In [266]:
from surprise import SVD
from surprise import Dataset
from surprise import KNNWithMeans,accuracy
from surprise.model_selection import cross_validate,train_test_split
import pandas as pd

Dictionaries we gonna use

In [267]:
dict_itens_by_id = dict()
dict_id_by_itens = dict()
watched = dict()
wasnt_watched = dict()

In [280]:
'''
Gettin all itens from the data base.
    Arranging into a dictonary adding key=id and value=item_name
'''
itens = open('data/ml-100k/u.item',encoding="ISO-8859-1")
item_data = itens.read().split('\n')
for info in item_data:
    split = info.split('|')
    if(split != ['']):
        id_item = (split[0])
        film = (split[1])
        dict_itens_by_id[id_item]=[film]

        dict_id_by_itens[film]=id_item
itens.close()

In [281]:
'''
Getting films that user have been watched
     Arranging into a dictonary adding key=id and value=item_name 
'''
base = open('data/ml-100k/u1.base',encoding="ISO-8859-1")
item_base = base.read().split('\n')
for usr_info in item_base:
    split = usr_info.split('\t')
    if(split != ['']):
        key = split[0]
        value = split[1]
        if(key in watched):
            watched[key].append(value)
        else:
            watched[key] = [value]
base.close()

In [282]:
'''
Getting films that user didnt watch
     Arranging into a dictonary adding key=id and value=item_name 
'''
films_list = set(dict_itens.keys())
for i in watched:
    films_list_user = set(watched[i])
    wasnt_watched[i] = films_list.difference(films_list_user)

K Nearest Neighbors <br></br>

To train our model, we gonna use KNN algorithm. Using a option to do the train using cosine, after some operations into a graph, the KNN will return the nexts neighbors witch those are much like an other

In [271]:
'''
Training our data using KNN
'''
data = Dataset.load_builtin('ml-100k')

trainset,testset = train_test_split(data,test_size =.20)

algo_KNN = KNNWithMeans(k=4, sim_options={'name': 'cosine', 'user_based': True})

algo_KNN.fit(trainset)
predictions_knn = algo_KNN.test(testset)
print('KNN RMSE: %.3f' % accuracy.rmse(predictions_knn,verbose = False))

Computing the cosine similarity matrix...
Done computing similarity matrix.
KNN RMSE: 1.034


Theoretically, as smaller the RMSE is, also smaller is the error. To our value, the error seems realy small, passing the impression that our prediction has no so much error.<br></br>

And this value is pretty much equaly from the  <a href="http://surpriselib.com/">Surprise.</a> 

So, taking that, looks our prediction to the films is gonna be great!


In [272]:
def get_5_films(uid):
    '''
    using the trained model for predict our filme users,
        this function will return the top 5 films to the parse
        user id
        Args:
        --------
        uid: Id from User
        Return:
        -------
        top_5: The 5 films most recomendaded to the uid
    '''
    predict = []
    films_from_user = list(wasnt_watched[uid])

    for i in films_from_user:
        predict.append((i,algo_KNN.predict(uid=uid,iid=str(i)).est))
    predict = sorted(predict, key=lambda x: x[1],reverse = True)
    top_5 = predict[:5]
    return top_5


In [273]:
def show_top_five(uid):
    '''
    Show the top five films to the user
    Args:
    --------
        uid:Id from user
    Return:
    --------
        top_five: list that contains the 5 itens to the user
    '''
    top_five = tuple()
    first_five = get_5_films(uid)
    for w in first_five:
        print(dict_itens_by_id[w[0]],w[1])

We also can return a recomendation by a film:

In [274]:
def show_equaly_film(film_name):
    '''
        That function will return a film depending of 
            the entry film
        Args:
        --------
            film_name: a film name and his date
        Return:
        --------
            the name of the closest film
    '''
    film_raw_id = dict_id_by_itens[film_name]
    film_inner_id = algo_KNN.trainset.to_inner_iid(film_raw_id)
    return(dict_itens_by_id[str(film_inner_id)])

Samples:<br></br>
    <p>show_top_five('230')</p>
    <p>show_equaly_film('Toy Story (1995)')</p>

In [277]:
show_top_five('230')

['Mrs. Brown (Her Majesty, Mrs. Brown) (1997)'] 5
['Titanic (1997)'] 5
['Babe (1995)'] 5
['Great Day in Harlem, A (1994)'] 5
['L.A. Confidential (1997)'] 5


In [278]:
show_equaly_film('Toy Story (1995)')

['Bad Boys (1995)']

Observation:<br></br>

As we see, my trainset and test set is no static. I always do the partition in train and test(80-20). So the resuls (hardely) will be the same twice.